In [2]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

In [3]:
# enter details of your AML workspace
subscription_id = "606a443f-9d6b-4284-8781-9285eddbdbc3"
resource_group = "mlops_rg"
workspace = "sample_ml_workspace"

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [5]:
import datetime

endpoint_name = "endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")

endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="This is a new online endpoint",
    auth_mode="key",
)

print(endpoint_name)

endpt-04131223429957


In [6]:
model = Model(path="../sklearn_regression_model.pkl")
import os 
print(os.path.exists("../sklearn_regression_model.pkl"))

True


In [7]:
env = Environment(
    conda_file="../conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

In [22]:
from pathlib import Path

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../",
        scoring_script="new-score.py",
    ),
    instance_type="Standard_F4S_V2",
    instance_count=1,
)

In [63]:
ml_client.online_endpoints.begin_delete(name=endpoint_name, local=True)

In [11]:
# Creating Local Endpoint
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

Updating local endpoint (endpt-04131223429957) .Done (0m 5s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': None, 'scoring_uri': None, 'openapi_uri': None, 'name': 'endpt-04131223429957', 'description': 'This is a new online endpoint', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': PosixPath('/home/azureuser/.azureml/inferencing/endpt-04131223429957'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f1a001beb90>, 'auth_mode': 'key', 'location': None, 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [60]:
for ep in ml_client.online_endpoints.list():
    print(ep.name, ep.provisioining_state)

In [10]:
!docker ps

CONTAINER ID   IMAGE                 COMMAND                CREATED          STATUS          PORTS     NAMES
8f7af0e807b3   localhost/c3:latest   "/usr/local/bin/ice"   14 minutes ago   Up 14 minutes             c3-progenitor


In [66]:
ml_client.online_deployments.begin_delete(
    name="blue",
    endpoint_name=endpoint_name,
    local=True
)

LocalEndpointNotFoundError: Local deployment (new-iris-endpoint / blue) does not exist.

In [12]:
# Create Local deployment
ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment, local=True
)

Creating local deployment (endpt-04131223429957 / blue) .
Building Docker image from Dockerfile
Step 1/6 : FROM mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest
... ---> 83631454f85b
Step 2/6 : RUN mkdir -p /var/azureml-app/
 ---> Running in d4cc4d801677
 ---> 62e7f64fe3d8
Step 3/6 : WORKDIR /var/azureml-app/
 ---> Running in 34d729d66787
. ---> b4e9010ab59d
Step 4/6 : COPY conda.yml /var/azureml-app/
 ---> 07d92dff15f6
Step 5/6 : RUN conda env create -n inf-conda-env --file conda.yml
 ---> Running in 61d160d1a6dc
Retrieving notices: - done
Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: ...working... done


==> WARNING: A newer version of conda exists. <==
    current version: 25.1.1
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



python-3.9.22        | 22.6 MB   |            |   0% 
scipy-1.10.1         | 14.4 MB   |            |   0% 

scikit-learn-1.2.2   | 7.3 MB    |            

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'endpt-04131223429957', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/tarunkw041/code/Users/tarunkw04/mlops_proj'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f19f25f1330>, 'model': Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': '7713d7a5680d37a33a7ac52530aec294', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/tarunkw041/code/Users/tarunkw04/mlops_proj'), 'creation_context': None, 'serialize': <msr

In [13]:
import os

print(os.path.exists("../new-score.py"))
model_dir = os.environ.get("AZUREML_MODEL_DIR")
print(model_dir)

True
None


In [14]:
ml_client.online_endpoints.get(name=endpoint_name, local=True)

ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Succeeded', 'scoring_uri': 'http://localhost:32770/score', 'openapi_uri': None, 'name': 'endpt-04131223429957', 'description': 'This is a new online endpoint', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/tarunkw041/code/Users/tarunkw04/mlops_proj'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f19f25f3d00>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [15]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, local=True, lines=50
)

'Azure ML Inferencing HTTP server v0.8.4.1\r\n\r\n\r\nServer Settings\r\n---------------\r\nEntry Script Name: /var/azureml-app/tarunkw04/new-score.py\r\nModel Directory: /var/azureml-app/azureml-models//7713d7a5680d37a33a7ac52530aec294/1\r\nConfig File: None\r\nWorker Count: 1\r\nWorker Timeout (seconds): 300\r\nServer Port: 31311\r\nHealth Port: 31311\r\nApplication Insights Enabled: false\r\nApplication Insights Key: None\r\nInferencing HTTP server version: azmlinfsrv/0.8.4.1\r\nCORS for the specified origins: None\r\nCreate dedicated endpoint for health: None\r\n\r\n\r\nServer Routes\r\n---------------\r\nLiveness Probe: GET   127.0.0.1:31311/\r\nScore:          POST  127.0.0.1:31311/score\r\n\r\n2025-04-13 12:28:46,754 I [20] gunicorn.error - Starting gunicorn 20.1.0\r\n2025-04-13 12:28:46,755 I [20] gunicorn.error - Listening at: http://0.0.0.0:31311 (20)\r\n2025-04-13 12:28:46,755 I [20] gunicorn.error - Using worker: sync\r\n2025-04-13 12:28:46,757 I [95] gunicorn.error - Booti

In [19]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file="../sample-request.json",
    local=True,
)

'[11055.977245525679, 4503.079536107787, 12266.841758291512]'

In [20]:
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpt-04131223429957.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://endpt-04131223429957.eastus.inference.ml.azure.com/swagger.json', 'name': 'endpt-04131223429957', 'description': 'This is a new online endpoint', 'tags': {}, 'properties': {'createdBy': 'Tarun Kumar', 'createdAt': '2025-04-13T12:35:48.330434+0000', 'lastModifiedAt': '2025-04-13T12:35:48.330434+0000', 'azureml.onlineendpointid': '/subscriptions/606a443f-9d6b-4284-8781-9285eddbdbc3/resourcegroups/mlops_rg/providers/microsoft.machinelearningservices/workspaces/sample_ml_workspace/onlineendpoints/endpt-04131223429957', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/606a443f-9d6b-4284-8781-9285eddbdbc3/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oeidp:268f9189-0238-4883-bf0e-df0de49c18b4:83908132-c4c7-4f30-a362-88001bce971b?api-versio

In [23]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Check: endpoint endpt-04131223429957 exists
Uploading tarunkw04 (0.19 MBs): 100%|██████████| 186021/186021 [00:00<00:00, 3099429.25it/s]




...................................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'endpt-04131223429957', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/606a443f-9d6b-4284-8781-9285eddbdbc3/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/odidp:268f9189-0238-4883-bf0e-df0de49c18b4:21057c2e-738f-4c8a-a24c-80b565817e9c?api-version=2023-04-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/606a443f-9d6b-4284-8781-9285eddbdbc3/resourceGroups/mlops_rg/providers/Microsoft.MachineLearningServices/workspaces/sample_ml_workspace/onlineEndpoints/endpt-04131223429957/deployments/blue', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/tarunkw041/code/Users/tarunkw04/mlops_proj', 'creation_context': <azure.ai.ml._restclient.v2023_04_01_preview.models._models_

In [24]:
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpt-04131223429957.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://endpt-04131223429957.eastus.inference.ml.azure.com/swagger.json', 'name': 'endpt-04131223429957', 'description': 'This is a new online endpoint', 'tags': {}, 'properties': {'createdBy': 'Tarun Kumar', 'createdAt': '2025-04-13T12:35:48.330434+0000', 'lastModifiedAt': '2025-04-13T12:55:36.947996+0000', 'azureml.onlineendpointid': '/subscriptions/606a443f-9d6b-4284-8781-9285eddbdbc3/resourcegroups/mlops_rg/providers/microsoft.machinelearningservices/workspaces/sample_ml_workspace/onlineendpoints/endpt-04131223429957', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/606a443f-9d6b-4284-8781-9285eddbdbc3/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oeidp:268f9189-0238-4883-bf0e-df0de49c18b4:aef14a31-919b-4de0-b33f-dbba0633e6e0?api-versio

In [4]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="blue",
    request_file="../sample-request.json",
)

NameError: name 'endpoint_name' is not defined